<a href="https://colab.research.google.com/github/FastBackendML/FastBackendML/blob/main/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.10 Использование CPU и GPU на примере простой НС

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t1 = torch.arange(1,6, dtype=torch.int32, device=device)
t2 = torch.ones(15, dtype=torch.float32, device=device).view(3,5)
t_res = t2.matmul(t1.float()).to(device='cpu')

In [ ]:
t_res

tensor([15., 15., 15.])

Task 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
targets = torch.rand(1000).to(device)
predict = torch.rand(1000).to(device)

In [ ]:
Q = torch.sum(torch.pow((targets - predict), 2)) / 1000

In [ ]:
Q = Q.to('cpu')

In [ ]:
Q

tensor(169.7121)

Task 3

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

w = torch.rand(1, 16).to(device) # вектор-строка
X = torch.rand(16, 16).to(device) # матрица 16 x 16
I = torch.eye(16).to(device) # единичная матрица 16 x 16
lm = 0.5 # значение лямбда

In [ ]:
Q = w.matmul((X.add(I.mul(lm)))).matmul(w.transpose(0,1))

In [ ]:
Q.to('cpu')

tensor([[27.8480]])

Task 4

In [ ]:
import torch
# все эти переменные в программе не менять, только тензоры разместить на device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

W = torch.rand(3) * 10 - 5
bias = torch.rand(1) * 100 - 50

batch_size = 8 # размер мини-батча
X = torch.empty(batch_size, 3).normal_(mean=-1.0, std=4.0).to(device)
predict = (X.to(device).mv(W.to(device)) + bias.to(device)).sigmoid()

# 1.11 Персептрон - возможности классификации образов

In [ ]:
import torch

# тензор X в программе не менять
batch_size = 32 # количество входных данных
X = torch.tensor(list(map(float, input().split())), dtype=torch.float32).view(batch_size, 2)

def line_coeff(x0, y0, x1, y1):
    w1 = y1 - y0
    w2 = -(x1 - x0)
    w0 = x1 * y0 - x0 * y1
    return -w0, -w1, -w2


def activate(x):
    return torch.sign(x)


w10, w11, w12 = line_coeff(-4, -6, -5, -3)
w20, w21, w22 = line_coeff(-5, -3, 1, 5)
w30, w31, w32 = line_coeff(1, 5, 10, 9)
w0, w1, w2, w3 = 2.5, 1, 1, 1


W1 = torch.tensor([(w11, w12), (w21, w22), (w31, w32)], dtype=torch.float32)
bias1 = torch.tensor([w10, w20, w30], dtype=torch.float32)
W2 = torch.tensor([w1, w2, w3], dtype=torch.float32)
bias2 = torch.tensor([w0], dtype=torch.float32)


X1_out = activate(X @ W1.permute(1, 0) + bias1)
predict = activate(X1_out @ W2 + bias2)

In [ ]:
import torch


# тензор X в программе не менять
batch_size = 32 # количество входных данных
X = torch.tensor(list(map(float, input().split())), dtype=torch.float32).view(batch_size, 2)

W1 = torch.tensor([[-1.8,  1.0],
                   [-0.75,  1.0]])
bias1 = torch.tensor([-2.4, -4.5])
# Веса выходного слоя сделаем равные (равный вклад двух нейронов скрытого слоя)
W2 = torch.tensor([1.0, 1.0])
# Если оба выхода скрытого слоя нулевые, нужно сместить их сумму ниже нуля
bias2 = torch.tensor([0.5])

# Функция активации общая для всех трёх нейронов
act = lambda z: torch.where(z > 0.0, torch.ones_like(z), -torch.ones_like(z))

# Находим выход скрытого слоя
h = act(X @ W1.t() + bias1)

# Находим выход выходного слоя
predict = act(h @ W2 + bias2)
# predict


# 2.1 Идея обучения НС градиентным алгоритмом

Task 1

In [ ]:
def func(x):
    return 0.5 * x + 0.2 * x ** 2 - 0.1 * x ** 3


def df(x):
    return 0.5 + 0.4 * x - 0.3 * x ** 2


# здесь продолжайте программу
n = 0.01
x0 = -4
N = 200
x = x0

for i in range(N):
    dif = df(x)
    x = x - n * dif

Task 2

In [ ]:
import torch

# Целевая функция
def func(x):
    return 0.1 * x**2 - torch.sin(x) + 5.

# Данные
coord_x = torch.arange(-5.0, 5.0, 0.1)
coord_y = func(coord_x).view(-1, 1)  # [N, 1]

# Подготовка признаков: [1, x, x^2, x^3]
X = torch.stack([torch.ones_like(coord_x), coord_x, coord_x**2, coord_x**3], dim=1)  # [N, 4]

# Начальные параметры
w = torch.zeros((4, 1), dtype=torch.float32)  # [4, 1]
eta = torch.tensor([0.1, 0.01, 0.001, 0.0001]).view(4, 1)  # [4, 1]
N = 200
sz = coord_x.size(0)

# Градиентный спуск
for i in range(N):
    y_pred = X @ w  # [N, 1]
    error = y_pred - coord_y  # [N, 1]
    grad = (2 / sz) * (X.T @ error)  # [4, 1]
    w = w - eta * grad  # Поэлементное обновление

# Итоговая ошибка
y_pred = X @ w
Q = torch.mean((y_pred - coord_y)**2).item()

# 2.3 Функции активации и потерь в PyTorch

Task 1

In [ ]:
import torch

# значения x, func, predict не менять
x = torch.arange(-3, 3, 0.1)
func = x ** 2 - 2 * torch.cos(x) - 5
predict = func + torch.empty_like(func).normal_(0, 0.5)

loss_func = torch.nn.MSELoss()
Q = loss_func(predict, func).float().item()

Q_mse = torch.sum((predict - func)**2) / len(predict)
Q_mse = Q_mse.float().item()

Task 2

In [ ]:
import torch

# значения predict, target не менять
batch_size = 8
target = torch.randint(0, 2, (batch_size, 1), dtype=torch.float32) # целевые значения
predict = torch.empty(batch_size, 1).normal_(0, 2.0) # прогнозные значения

loss_func = torch.nn.BCEWithLogitsLoss()
Q = loss_func(predict, target).float().item()
sig_preds = torch.sigmoid(predict)
eps = 1e-7  # защита от log(0)
Q_bce = -torch.sum(target * torch.log(sig_preds + eps) + (1 - target) * torch.log(1 - sig_preds) + eps) / batch_size
Q_bce = Q_bce.float().item()
print(Q_bce, Q)

0.9961713552474976 0.9961714744567871


# 2.4 Автоматическое дифференцирование

Task 1

In [ ]:
import torch

d = float(input()) # значение d в программе не менять
t = torch.tensor([d], requires_grad=True)
f = 3 * t**2 + 5 * t - 2
f.backward()
print(f)
print(t.data, t.grad)

50
tensor([7748.], grad_fn=<SubBackward0>)
tensor([50.]) tensor([305.])


Task 2

In [ ]:
import torch

g = float(input()) # значение g в программе не менять
d = float(input()) # значение d в программе не менять

t = torch.tensor([d], requires_grad=True)

f = -(g * t**2) / 2 + g * torch.exp(-t) + 1.5

f.backward()

print(t.data, t.grad)
print(f)

50
30
tensor([30.]) tensor([-1500.])
tensor([-22498.5000], grad_fn=<AddBackward0>)


Task 3

In [ ]:
import torch

xx = float(input()) # значение xx в программе не менять
x = torch.tensor(xx, dtype=torch.float32)

x0, x1, x2, x3 = map(float, input().split()) # переменные x0, x1, x2, x3 в программе не менять

w0 = torch.tensor(x0, dtype=torch.float32, requires_grad=True)
w1 = torch.tensor(x1, dtype=torch.float32, requires_grad=True)
w2 = torch.tensor(x2, dtype=torch.float32, requires_grad=True)
w3 = torch.tensor(x3, dtype=torch.float32, requires_grad=True)

a = w0 + w1 * x + w2 * x**2 + w3 * x**3
f = 0.5 * x**2 - 2 * x - 5
L = (a - f)**2
L.backward()